# Terms, Formulas and Interpretations

Now we have all the elements to formally define ```Tarski``` languages:

**Definition** (Many-Sorted First-Order Language). A _many-sorted_ _first-order_ language ${\cal L}$ is made up of:
 - A non-empty set $T$ of _sorts_
 - An _infinite number_ of _variables_ $x_{1}^{\tau}, x_{2}^{\tau}, \ldots$ for each short $\tau \in T$
 - For each $n \geq 0$ and each tuple $(\tau_1, \ldots, \tau_{n+1}) \in T^{n+1}$ of sorts, a (possibly empty) set of _function_ symbols, each of which is said to have _arity_ and _type_ $(\tau_1, \ldots, \tau_{n+1})$
 - For each $n \geq 0$ and each tuple $(\tau_1, \ldots, \tau_{n+1}) \in T^{n}$ of sorts, a (possibly empty) set of _relation_ symbols (predicates), each of which is said to have _arity_ and _type_ $(\tau_1, \ldots, \tau_{n})$

Continuing with our ```Blocks World``` themed example

In [1]:
import tarski
from tarski.symbols import *
from tarski.theories import Theory

# 1. Create language used to describe world states and transitions
bw = tarski.language(theories=[Theory.EQUALITY, Theory.ARITHMETIC])

# 2. Define sorts
place = bw.sort('place')
block = bw.sort('block', place)

# 3. Define functions
loc = bw.function( 'loc', block, place )
looking_at = bw.function( 'looking_at', block )

# 4. Define predicates
clear = bw.predicate( 'clear', block)

We introduce the function $width(b)$ for blocks $b$, this will allow us to specify Hanoi Towers like tasks

In [2]:
width = bw.function('width', block, bw.Real)

_Constants_ are 0-arity functions, whose sort $\tau$ is a set with one single element. Hence, we handle them separately, as we specialise their representation

In [3]:
# 5. Define constants
b1, b2, b3, b4 = [ bw.constant('b_{}'.format(k), block) for k in (1,2,3,4) ]
table = bw.constant('table', place)

## (First-Order) Terms

Combinations of variables, functions and constants are called _terms_, and the rules for constructing them are given inductively:

**Definition** (First-Order Terms). A term $t$ can be:

 - Any variable $x^{\tau}$ of the language can be a term $t$ with type $\tau$
 - Any constant symbol of the language with type $\tau$ is a term with the same type
 - If $t_1, \ldots, t_n$ are terms with respective types $\tau_1, \ldots, \tau_n$ and $f$ is a _function_ symbol with type $(\tau_1, \ldots, \tau_n, \tau{n+1})$ then $f(t_1,\ldots,t_n)$ is a term with type $\tau_{n+1}$.

Terms are implemented as Python objects. Every constant symbol is an instance of ```Term```

In [4]:
from tarski import Term

isinstance(b1,Term)

True

Function symbols allow to nest terms, thus 

In [5]:
t1 = loc(b1)
isinstance(t1,Term)

True

In [6]:
x = bw.variable('x', block)
t2 = loc(x)
isinstance(t2,Term)

True

In [7]:
t3 = loc(looking_at())
isinstance(t3,Term)

True

are all terms. ```Tarski``` textual representation of variables is a bit different

In [8]:
print('{}, type: {}'.format(t1, t1.sort))
print('{}, type: {}'.format(t2, t2.sort))
print('{}, type: {}'.format(t3, t3.sort))

loc(b_1), type: Sort(place)
loc(x/block), type: Sort(place)
loc(looking_at()), type: Sort(place)


in order to make distinct variables from constants, the former are printed with the prefix ```?```. 

## Formulas

Formulas (statements that can be either ```True``` or ```False```) are defined also inductively as follows:

**Definition** (First-Order Formulas).

 - If $t_1$ and $t_2$ are two terms with the same type, then $t_1 = t_2$ is an _atomic formula_.
 
 - If $t_1,\ldots,t_n$ are terms with respective types $\tau_1,\ldots,\tau_n$, and $R$ is a relation symbol with type $(\tau_1,\ldots,\tau_n)$, then $R(t_1,\ldots,t_n)$ is an atomic formula too.
 
 - If $\phi_1$ and $\phi_2$ are formulas then $\neg \phi_1$, $\phi_1 \lor \phi_2$ and $\phi_1 \land \phi_2$ are also formulas.
 
 - If $\phi$ is a formula, then $\exists_t x^{\tau}\, \phi$ and $\forall_t x^{\tau}\, \phi$ are also formulas.

Quantification happens over a certain sort, i.e. for each sort $\tau$ $\in$ $T$ there are universal and existential quantifier symbols $\forall_{\tau}$ and $\exists_{\tau}$, which may be applied to variables of the same sort.

Formulas without existential ($\exists$) or universal ($\forall$) quantifiers are called _quantifier free_.

### Examples

We can define the formula $t_1 = t_3$ - terms $t_1$ and $t_3$ are equal - with the following statement

In [9]:
tau = t1 == t3

The ```str()``` method is overwritten for every term and formula class, returning a string representation of expression, which gives insight into how Tarski represents internally formulas and expressions

In [10]:
str(tau)

'=(loc(b_1),loc(looking_at()))'

We need a new variable so we can make general statements about more than one block

In [11]:
y = bw.variable('y', block)

Now we can state properties of states like _for every block x, x cannot be wider than the place below_

$$
\forall x,y\, loc(x) = y \supset width(x) < width(y)
$$

which can be written as

In [12]:
phi = forall( x, y, implies( loc(x) == y, width(x) < width(y) ) )

which is represented internally

In [13]:
str(phi)

'forall x/block, y/block : ((not (=(loc(x/block),y/block)) or <(width(x/block),width(y/block))))'

It's worth noting that Tarski will always try to simplify formulas. For instance, the sub-formula 

$$ 
loc(x) = y \supset width(x) < width(y)
$$

was transformed into the disjunction

$$
loc(x) \neq y \lor width(x) < width(y)
$$

using the transformation 

$$
p \supset q \equiv \neg p \lor q
$$

We can use the operator ```>``` instead of the function ```implies()```, if a more concise syntax is preferred.

In [14]:
phi = forall( x, y, (loc(x) == y) > (width(x) < width(y)) )

We can write the conjunctive  formula

$$
loc(b1) \neq loc(b2) \land loc(b1) \neq loc(b3)
$$

in several ways. One is using the ```land()``` function

In [15]:
phi = land( loc(b1) != loc(b2), loc(b1) != loc(b3))

or the operator ```&```

In [16]:
phi = (loc(b1) != loc(b2)) & (loc(b1) != loc(b3))

Another state invariant like 

$$
loc(b1) = b2 \lor loc(b1) = b3
$$

can be written as

In [17]:
phi = lor( loc(b1) == b2, loc(b1) == b3 )

or

In [18]:
phi = (loc(b1)==b2) | (loc(b1)==b3)

Finally, the formula 

$$
loc(b1) = b2 \supset \neg clear(b2)
$$

can be written as 

In [19]:
phi=implies( loc(b1) == b2, neg(clear(b2)))
str(phi)

'(not (=(loc(b_1),b_2)) or not (clear(b_2)))'

or, alternatively the ```~``` unary operator can be used instead of ```neg(...)```

In [20]:
phi = implies( loc(b1) == b2, ~clear(b2))
str(phi)

'(not (=(loc(b_1),b_2)) or not (clear(b_2)))'

### Next: [Interpretations](004_interpretations.ipynb)